In [16]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [17]:
# association recognition - cued recall
with open("../Data/simuS1_group1_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
df_study = df_study.loc[df_study.session < 100]
df_test = df_test.loc[df_test.session < 100]

In [18]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session,pair_idx
0,1118,147,PRAIRIE,BOMB,0,0
1,581,634,FLOOR,GHOST,0,1
2,1087,710,PLAYGROUND,HIP,0,2
3,1458,679,THIMBLE,HAMBURGER,0,3
4,745,589,ISLAND,FOLLOWER,0,4
...,...,...,...,...,...,...
4795,514,793,EARRING,LADDER,99,43
4796,740,906,INSECT,MONARCH,99,44
4797,1435,116,TAR,BEETLE,99,45
4798,533,21,ENCYCLOPEDIA,ANCHOR,99,46


In [19]:
df_test

,test_itemno1,test_itemno2,test_item1,test_item2,correct_ans,order,pair_idx,test,session
0,1354,563,SPIT,FIG,1,1,38,1,0
1,634,581,GHOST,FLOOR,1,1,1,1,0
2,262,1330,CAULIFLOWER,SMOG,0,0,-1,1,0
3,108,1356,BEAST,SPOOL,1,1,39,1,0
4,247,1315,CARPET,SKETCH,1,0,19,1,0
...,...,...,...,...,...,...,...,...,...
5995,63,-1,AWARD,-1,340,1,18,2,99
5996,415,-1,CROWN,-1,1089,0,15,2,99
5997,780,-1,KITTEN,-1,1513,0,4,2,99
5998,1275,-1,SHARK,-1,1343,0,16,2,99


### Run CMR-IA

In [20]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [21]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_cue = 0.5,
    beta_rec = 0,
    beta_rec_new = 0.1, # beta for retrieved item
    beta_rec_post = 0.1,
    gamma_fc = 0.5,
    gamma_cf = 0.5,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.70,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.02,
    lamb = 0,
    eta = 0.01,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 96,
    d_ass = 1,
    learn_while_retrieving = True,
    omega = 2,
    alpha = 1
)
params

{'beta_enc': 0.5,
 'beta_rec': 0,
 'beta_cue': 0.5,
 'beta_rec_new': 0.1,
 'beta_rec_post': 0.1,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0.01,
 'omega': 2,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.7,
 'd_ass': 1,
 'lamb': 0,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 96,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5}

In [22]:
df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params,df_study, df_test ,s_mat)
df_simu

CMR2 Time: 7.061145305633545


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim
0,0,1354,563,1.0,559.779846,0.762257
1,0,634,581,0.0,798.940796,0.596022
2,0,262,1330,0.0,1418.918213,0.327628
3,0,108,1356,1.0,517.093201,0.799323
4,0,247,1315,1.0,597.301880,0.731940
...,...,...,...,...,...,...
5995,99,63,-1,340.0,1630.000000,0.762963
5996,99,415,-1,1089.0,1440.000000,0.830522
5997,99,780,-1,1513.0,1500.000000,0.753889
5998,99,1275,-1,1343.0,1430.000000,0.831301


In [23]:
df_simu = df_simu.merge(df_test,on=['session','test_itemno1','test_itemno2'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_item1,test_item2,correct_ans,order,pair_idx,test,correct
0,0,1354,563,1.0,559.779846,0.762257,SPIT,FIG,1,1,38,1,True
1,0,634,581,0.0,798.940796,0.596022,GHOST,FLOOR,1,1,1,1,False
2,0,262,1330,0.0,1418.918213,0.327628,CAULIFLOWER,SMOG,0,0,-1,1,True
3,0,108,1356,1.0,517.093201,0.799323,BEAST,SPOOL,1,1,39,1,True
4,0,247,1315,1.0,597.301880,0.731940,CARPET,SKETCH,1,0,19,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,99,63,-1,340.0,1630.000000,0.762963,AWARD,-1,340,1,18,2,True
5996,99,415,-1,1089.0,1440.000000,0.830522,CROWN,-1,1089,0,15,2,True
5997,99,780,-1,1513.0,1500.000000,0.753889,KITTEN,-1,1513,0,4,2,True
5998,99,1275,-1,1343.0,1430.000000,0.831301,SHARK,-1,1343,0,16,2,True


In [24]:
# get f_in
sessions = np.unique(df_simu.session)
tmp_corr_fin = []
tmp_omax_fin = []
test1_num = 40

for sess in sessions:
    df_tmp = df_study.loc[df_study.session == sess]
    tmp1 = df_tmp.study_itemno1.to_numpy()
    tmp2 = df_tmp.study_itemno2.to_numpy()
    tmp = np.concatenate((tmp1, tmp2))
    tmp = np.sort(tmp)

    tmp_corr = df_simu.loc[df_simu.session == sess,"correct_ans"][test1_num:]
    corrid = np.searchsorted(tmp, tmp_corr)

    corr_fin = [f_in_dif[sess][i][id] for i, id in enumerate(corrid)]
    omax_fin = [np.max(np.delete(f_in_dif[sess][i], id)) for i, id in enumerate(corrid)]

    tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
    tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

df_simu['corr_fin'] = tmp_corr_fin
df_simu['omax_fin'] = tmp_omax_fin
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_item1,test_item2,correct_ans,order,pair_idx,test,correct,corr_fin,omax_fin
0,0,1354,563,1.0,559.779846,0.762257,SPIT,FIG,1,1,38,1,True,-1.000000,-1.000000
1,0,634,581,0.0,798.940796,0.596022,GHOST,FLOOR,1,1,1,1,False,-1.000000,-1.000000
2,0,262,1330,0.0,1418.918213,0.327628,CAULIFLOWER,SMOG,0,0,-1,1,True,-1.000000,-1.000000
3,0,108,1356,1.0,517.093201,0.799323,BEAST,SPOOL,1,1,39,1,True,-1.000000,-1.000000
4,0,247,1315,1.0,597.301880,0.731940,CARPET,SKETCH,1,0,19,1,True,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,99,63,-1,340.0,1630.000000,0.762963,AWARD,-1,340,1,18,2,True,-0.311118,-0.500929
5996,99,415,-1,1089.0,1440.000000,0.830522,CROWN,-1,1089,0,15,2,True,-0.237509,-0.427311
5997,99,780,-1,1513.0,1500.000000,0.753889,KITTEN,-1,1513,0,4,2,True,-0.334051,-0.478407
5998,99,1275,-1,1343.0,1430.000000,0.831301,SHARK,-1,1343,0,16,2,True,-0.250209,-0.442675


In [25]:
# recognition performance
df_recog = df_simu.query("test==1")
df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")

,Yes rate
correct_ans,
0,0.050
1,0.849


In [26]:
# cued recall performance
df_cr = df_simu.query("test==2")
df_cr.correct.mean()

0.634

In [27]:
def get_pair(df_tmp):
    df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
    df_tmp_pair.columns = ["test1","test2"]
    df_tmp_pair.reset_index(inplace=True)
    return df_tmp_pair

df_simu_p = df_simu.query("pair_idx >= 0")
df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
df_pair

,session,level_1,pair_idx,test1,test2
0,0,0,1,0.0,1.0
1,0,1,3,1.0,0.0
2,0,2,4,1.0,0.0
3,0,3,9,1.0,1.0
4,0,4,12,1.0,1.0
...,...,...,...,...,...
1995,99,15,25,1.0,0.0
1996,99,16,26,1.0,1.0
1997,99,17,34,1.0,0.0
1998,99,18,36,1.0,1.0


In [28]:
df_tab = pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all')
df_tab

test1,0.0,1.0
test2,,
0.0,0.0545,0.3115
1.0,0.0965,0.5375


In [29]:
def Yule_Q(A, B, C, D):
    return (A * D - B * C) / (A * D + B * C)
Yule_Q(df_tab[1][1],df_tab[0][1],df_tab[1][0],df_tab[0][0])

-0.012905725862838741

In [30]:
# add 0.5 to each cell
Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)

0.08612233782207736